In [0]:
import pandas as pd
import numpy as np

from itertools import chain
import matplotlib.pyplot as plt
import os
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:

#Import Training and Testing Data
data = pd.read_csv('/content/gdrive/My Drive/NLP/Dataset/train_E6oV3lV.csv')

### Tokenization

In [8]:
tokenized_single_posts = [nltk.tokenize.word_tokenize(i) for i in data['tweet']]
len(tokenized_single_posts)

31962

In [0]:
leng = []
for i in range(len(tokenized_single_posts)):
    length = len(tokenized_single_posts[i])
    leng.append(length)

### stopwords removal

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
stp_removed = []

for i in range (len(tokenized_single_posts)):
    stp = [word for word in tokenized_single_posts[i] if word not in stopwords.words('english')]
    stp_removed.append(stp)
    

In [13]:
len(stp_removed)

31962

### Lemmatize the stop removed posts

In [15]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
words_lemma = []
lemma = nltk.WordNetLemmatizer()
for i in range(len(stp_removed)):
    words = [lemma.lemmatize(word) for word in stp_removed[i]]
    words_lemma.append(words)

In [17]:
len(words_lemma)

31962

### Remove all digit words

In [0]:
words_noNum = []
for i in range(len(words_lemma)):
    words = [word for word in words_lemma[i] if word.isdigit() == False]
    words_noNum.append(words)

In [19]:
len(words_noNum)

31962

### Remove single character words

In [0]:
words_nonSingle = []
for i in range(len(words_noNum)):
    words = [word for word in words_noNum[i] if len(word) > 1]
    words_nonSingle.append(words)

In [21]:
len(words_nonSingle)

31962

### Remove non-alphabetic words

In [0]:
words_alpha = []
for i in range(len(words_nonSingle)):
    words = [word for word in words_nonSingle[i] if word.isalpha()]
    words_alpha.append(words)

In [23]:
len(words_alpha)

31962

In [0]:
data['words_count'] = [len(i) for i in words_alpha]

In [26]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
noun_freq = []
verb_freq = []
adjective_freq = []
adverb_freq = []
for i in range(len(words_alpha)):
    word_pos_tag = nltk.pos_tag(words_alpha[i])
    count_noun = 0
    count_verb = 0
    count_adjective = 0
    count_adverb = 0
    for j in range(len(word_pos_tag)):
        if word_pos_tag[j][1] == "NN":
            count_noun += 1
        if word_pos_tag[j][1] == 'VB':
            count_verb += 1
        if word_pos_tag[j][1] == 'JJ':
            count_adjective += 1
        if word_pos_tag[j][1] == 'RB':
            count_adverb += 1
    noun_freq.append(count_noun/(len(words_alpha[i]) + 1))
    verb_freq.append(count_verb/(len(words_alpha[i])+1))
    adjective_freq.append(count_adjective/(len(words_alpha[i])+1))
    adverb_freq.append(count_adverb/(len(words_alpha[i])+1))

In [0]:
freq_dict = {'noun_freq' : noun_freq, 'verb_freq' : verb_freq, 'adjective_freq' : adjective_freq, 'adverb_freq' : adverb_freq}

In [0]:
data = data.join(pd.DataFrame(freq_dict))

In [31]:
data.tail()

,id,label,tweet,words_count,noun_freq,verb_freq,adjective_freq,adverb_freq
31957,31958,0,ate @user isz that youuu?ðððððð...,4,0.600000,0.200000,0.000000,0.0
31958,31959,0,to see nina turner on the airwaves trying to...,14,0.400000,0.066667,0.133333,0.0
31959,31960,0,listening to sad songs on a monday morning otw...,8,0.666667,0.000000,0.111111,0.0
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,...",8,0.666667,0.000000,0.111111,0.0
31961,31962,0,thank you @user for you follow,3,0.750000,0.000000,0.000000,0.0


In [38]:
features = data.drop(['tweet','id'],axis = 1)
features.head()

,label,words_count,noun_freq,verb_freq,adjective_freq,adverb_freq
0,0,8,0.444444,0.111111,0.222222,0.000000
1,0,14,0.400000,0.066667,0.066667,0.066667
2,0,2,0.666667,0.000000,0.000000,0.000000
3,0,4,0.600000,0.200000,0.000000,0.000000
4,0,3,0.500000,0.000000,0.000000,0.250000


In [0]:
X = features.iloc[:,1:6]
y = features['label']

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [53]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=42)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print(y_pred)

[0 0 0 ... 0 0 0]


In [54]:
print("ACCURACY : ",clf.score(X_test,y_test))

ACCURACY :  0.9296549108835799


In [49]:
from sklearn.metrics import confusion_matrix
print("CONFUSION MATRIX\n", confusion_matrix(y_test, y_pred))

CONFUSION MATRIX
 [[9806    0]
 [ 742    0]]


In [50]:
from sklearn.metrics import f1_score
print("F1 SCORE :",f1_score(y_test, y_pred, average='weighted'),"\n")

F1 SCORE : 0.8957645726760719 



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
